In [1]:
from bs4 import BeautifulSoup
import urllib.request
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np

In [3]:
import itertools

In [4]:
# scrap dari digilib

In [5]:
docs_num = 400

In [6]:
digilib_url = 'http://digilib.its.ac.id/publisher/51100/'

In [7]:
paper = []

In [8]:
for i in range(0,docs_num,20):
    page = urllib.request.urlopen(digilib_url)
    soup = BeautifulSoup(page, 'html.parser')
    docs = soup.find_all('span', attrs={'class': 'style5'})
    link = []
    for x in docs:
        try:
            link.append(x.find('a').get('href'))
        except:
            pass
    for x in tqdm(link[:20], desc='paper', unit='paper'):
        page = urllib.request.urlopen(x)
        soup = BeautifulSoup(page, 'html.parser')
        try:
            title = soup.find('h2', attrs={'class': 'isi'}).find('i').getText()
            abstract = soup.find('span', attrs={'class': 'teks'}).find('p').getText()
            paper.append([x, title, abstract])
        except:
            pass

paper: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  6.90paper/s]


In [9]:
len(paper)

360

In [10]:
# preprocessing

In [11]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
stemmer = StemmerFactory().create_stemmer()

In [12]:
words = []
processed_paper = []

In [13]:
for x in tqdm(paper, desc='paper', unit='paper'):
    text = x[2]
    text = text.lower()
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    text = text.translate(remove_punctuation_map)
    text = stopword.remove(text)
    text = text.split()
    text = [stemmer.stem(x) for x in text]
    processed_paper.append(' '.join(text))
    text = list(set(text))
    words += text

paper: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 360/360 [00:22<00:00, 15.85paper/s]


In [14]:
len(words)

29320

In [15]:
# generate thesaurus

In [16]:
thesaurus = {}

In [17]:
words = list(set(words))

In [18]:
for x in tqdm(words, desc='word', unit='word'):
    name = x
    data = { "q": name }
    encoded_data = urllib.parse.urlencode(data).encode("utf-8")
    content = urllib.request.urlopen("http://www.sinonimkata.com/search.php", encoded_data)
    soup = BeautifulSoup(content, 'html.parser')
    try:
        synonym = soup.find('td', attrs={'width': '90%'}).find_all('a')
        synonym = [x.getText() for x in synonym]
        thesaurus[x] = [x] + synonym
    except:
        thesaurus[x] = [name]

word: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 502/502 [07:22<00:00,  1.14word/s]


In [19]:
# tf_idf

In [20]:
vectorizer = TfidfVectorizer(use_idf=True)

In [38]:
query = 'pengenalan kendaraan'

In [39]:
query = query.lower()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
query = query.translate(remove_punctuation_map)
query = stopword.remove(query)
query = query.split()
query = [stemmer.stem(x) for x in query]

In [40]:
query

['kenal', 'kendara']

In [41]:
product_query = []
list_synonym = []

In [42]:
for x in query:
    if x in words:
        list_synonym.append(thesaurus[x])
    else:
        name = x
        data = { "q": name }
        encoded_data = urllib.parse.urlencode(data).encode("utf-8")
        content = urllib.request.urlopen("http://www.sinonimkata.com/search.php", encoded_data)
        soup = BeautifulSoup(content, 'html.parser')
        try:
            synonym = soup.find('td', attrs={'width': '90%'}).find_all('a')
            synonym = [x.getText() for x in synonym]
            thesaurus[x] = [x] + synonym
            list_synonym.append(thesaurus[x])
        except:
            list_synonym.append([x])

In [122]:
qs = []

In [123]:
for x in itertools.product(*list_synonym):
    x = [stemmer.stem(y) for y in x]
    qs.append([' '.join(x)])

In [135]:
qs

[['kenal kendara'],
 ['kenal bawa'],
 ['kenal naik'],
 ['kenal kemudi'],
 ['kenal kendali'],
 ['kenal gandar'],
 ['kenal jalan'],
 ['kenal tumpang'],
 ['kenal tunggang'],
 ['kenal setir'],
 ['ingat kendara'],
 ['ingat bawa'],
 ['ingat naik'],
 ['ingat kemudi'],
 ['ingat kendali'],
 ['ingat gandar'],
 ['ingat jalan'],
 ['ingat tumpang'],
 ['ingat tunggang'],
 ['ingat setir'],
 ['nali kendara'],
 ['nali bawa'],
 ['nali naik'],
 ['nali kemudi'],
 ['nali kendali'],
 ['nali gandar'],
 ['nali jalan'],
 ['nali tumpang'],
 ['nali tunggang'],
 ['nali setir'],
 ['tahu kendara'],
 ['tahu bawa'],
 ['tahu naik'],
 ['tahu kemudi'],
 ['tahu kendali'],
 ['tahu gandar'],
 ['tahu jalan'],
 ['tahu tumpang'],
 ['tahu tunggang'],
 ['tahu setir']]

In [129]:
max_result = []

In [130]:
for x in qs:
    paper_tfidf = vectorizer.fit_transform(x + processed_paper)
    q = paper_tfidf[0]
    result = cosine_similarity(paper_tfidf, q)
    idx = np.argsort(-result,axis=0).flatten()    
    final = [[num, y[0], x] for num, y in enumerate(result) if y[0] > 0.0]
    max_result += final

In [131]:
max_result = sorted(max_result, key=lambda x: x[1], reverse=True)

In [132]:
set_result = set()
new_result = []
for item in max_result:
    if item[0] not in set_result:
        set_result.add(item[0])
        new_result.append(item)
    else:
        pass
new_result

[[0, 1.0000000000000002, ['kenal kendara']],
 [12, 0.14360531244025423, ['kenal naik']],
 [13, 0.14360531244025423, ['kenal naik']],
 [30, 0.14360531244025423, ['kenal naik']],
 [31, 0.14360531244025423, ['kenal naik']],
 [48, 0.14360531244025423, ['kenal naik']],
 [49, 0.14360531244025423, ['kenal naik']],
 [66, 0.14360531244025423, ['kenal naik']],
 [67, 0.14360531244025423, ['kenal naik']],
 [84, 0.14360531244025423, ['kenal naik']],
 [85, 0.14360531244025423, ['kenal naik']],
 [102, 0.14360531244025423, ['kenal naik']],
 [103, 0.14360531244025423, ['kenal naik']],
 [120, 0.14360531244025423, ['kenal naik']],
 [121, 0.14360531244025423, ['kenal naik']],
 [138, 0.14360531244025423, ['kenal naik']],
 [139, 0.14360531244025423, ['kenal naik']],
 [156, 0.14360531244025423, ['kenal naik']],
 [157, 0.14360531244025423, ['kenal naik']],
 [174, 0.14360531244025423, ['kenal naik']],
 [175, 0.14360531244025423, ['kenal naik']],
 [192, 0.14360531244025423, ['kenal naik']],
 [193, 0.14360531244

In [133]:
for x in new_result[1:5]: 
    print('Result', x[0]+1) 
    print('QUERY', x[2]) 
    print(paper[x[0]-1][1]) 
    print(paper[x[0]-1][2])
    print()

Result 13
QUERY ['kenal naik']
IMPLEMENTASI ENAM MODUL MINI GAMES DENGAN MENERAPKAN BUILDER PATTERN UNTUK SISTEM PERMAINAN HEART MEISTER PADA UNITY 2D
Heart Meister merupakan permainan sosial pada perangkat
smartphone. Permainan Heart Meister bergenre Role Play Game RPG
dengan tema fantasi. Permainan ini berfokus pada pertarungan antar pet
yang dimiliki pemain. Setiap pet memiliki atribut sebagai indikator
kekuatannya. Level dari masing-masing atribut bisa dinaikkan dengan
menyelesaikan dungeon yang ada. Akan tetapi terkadang pemain ingin
menaikkan level dari atribut tersebut secepat mungkin untuk menjadikan
pet-nya semakin kuat. Masalah lain yaitu pet yang bukan merupakan pet
utama dari pemain akan susah dinaikkan level atributnya karena jarang
atau tidak pernah dipakai. Dengan demikian dibutuhkan suatu sistem
untuk menaikkan level atribut dari pet secara cepat dan menarik.
Modul mini game ini dibuat dengan tujuan untuk memenuhi
kebutuhan penaikan level atribut dari pet pada permainan